In [25]:
%pwd
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import csv

labels = [-1, 'CN', 'EMCI', 'LMCI', 'AD']

In [69]:
#Loading in data
df = pd.read_csv('Data/ADNI_Gene_Expression_Profile.csv', header=2)
df = df[6:]
df = df.drop(df.columns[[1, -1]], axis=1)
df = df.rename({'SubjectID':'ProbeSet', 'Unnamed: 2':'Gene'}, axis=1)
df = df.set_index(['Gene', 'ProbeSet'])
df = df.loc[df.index.dropna()]

dfanno = pd.read_csv('Data/ADNIMERGE.csv')
dfanno = pd.Series(dfanno['DX_bl'].values, index=dfanno['PTID'])
dfanno = dfanno.groupby(dfanno.index).last()
dfanno = dfanno.loc[df.columns]

In [70]:
#Load Genes
with open("genes.txt", "rb") as fp:   # Unpickling
    genes = pickle.load(fp)

In [71]:
#Choose Genes
dfanno = dfanno.loc[df.columns]
dftop = df.loc[genes]
dftop = dftop.groupby(dftop.index).first()
dftop = dftop.astype(float)

In [72]:
#Standardize
dftop = dftop.astype(float)
dftop = dftop.sub(dftop.mean(1), axis=0).div(dftop.std(1), axis=0)
X = dftop.T
y = dfanno
X.columns = pd.MultiIndex.from_tuples(X.columns.values).droplevel(1) #keep genes get rid of probes

In [66]:
AD = y.replace('AD', 0).replace('LMCI', 3).replace('EMCI', 2).replace('CN', 1)

In [67]:
BINARY = y.replace('AD', 0).replace('LMCI', 1).replace('EMCI', 1).replace('CN', 1)

In [82]:
print(X.T) #I think this is the format you want so just transpose X

         116_S_1249  037_S_4410  006_S_4153  116_S_1232  099_S_4205  \
ABCA1      1.822807   -0.779124    0.968616   -0.303925    0.381177   
ABCA4     -0.652624   -2.129883   -1.531127    0.392746   -1.462417   
ABCG4     -1.094384   -0.851499    0.197762   -0.754346   -0.584327   
ABHD12B    0.241881   -0.745238   -0.308628   -0.058684    1.238490   
ABI1       1.338883    0.891722    0.370033   -0.927762    1.292625   
...             ...         ...         ...         ...         ...   
ZNF836     0.678103   -0.856296    0.936616    0.890751    0.265316   
ZNF85     -1.240764   -0.141903    0.299064    1.664638   -1.240764   
ZSCAN23    0.062548   -1.229122   -0.951914   -0.002331    0.027159   
ZSCAN4     0.616652   -1.436080   -0.581359   -0.245075   -0.903630   
ZWILCH     0.074735    0.988636    0.288067   -0.999823    1.365259   

         007_S_4467  128_S_0205  003_S_2374  036_S_4491  098_S_4059  ...  \
ABCA1      0.378262    0.732474    2.281972    0.972989    0.628980  ..

In [8]:
#Just format the diagnosis values using AD (0, 1, 2, 3) or Binary (0, 1)

In [9]:
X.to_csv('Data/GSEAgene.gct.txt', quoting=csv.QUOTE_MINIMAL)

In [10]:
AD.to_csv('Data/GSEAdiagnosis.cls')

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [ ]:
BINARY.to_csv('Data/GSEAdiagnosis_binary.cls')